In [1]:
import tensorflow as tf
print(tf.__version__)

tf.enable_eager_execution()

1.14.0


In [2]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

In [4]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())
    
for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())

In [5]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [6]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [7]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

b'this was a wonderfully clever and entertaining movie that i shall never tire of watching many many times the casting was magnificent in matching up the young with the older characters there are those of us out here who really do appreciate good actors and an intelligent story format as for <OOV> <OOV> she is beautiful and a gift to any kind of production in which she stars i always make a point to see <OOV> <OOV> in all her performances she is a superb actress and a pleasure to watch as each transformation of her character comes to life i can only be grateful when i see such an outstanding picture for most of the motion pictures made more
b'This was a wonderfully clever and entertaining movie that I shall never tire of watching many, many times. The casting was magnificent in matching up the young with the older characters. There are those of us out here who really do appreciate good actors and an intelligent story format. As for Judi Dench, she is beautiful and a gift to any kind of

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') 
])

In [13]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [14]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - ETA: 1:38 - loss: 1.1418e-06 - acc: 1.000 - ETA: 15s - loss: 2.6357e-06 - acc: 1.000 - ETA: 10s - loss: 2.2848e-06 - acc: 1.00 - ETA: 8s - loss: 2.1824e-06 - acc: 1.0000 - ETA: 7s - loss: 2.2002e-06 - acc: 1.000 - ETA: 7s - loss: 2.1537e-06 - acc: 1.000 - ETA: 6s - loss: 2.1675e-06 - acc: 1.000 - ETA: 6s - loss: 2.1250e-06 - acc: 1.000 - ETA: 6s - loss: 2.0971e-06 - acc: 1.000 - ETA: 6s - loss: 2.0333e-06 - acc: 1.000 - ETA: 5s - loss: 1.9630e-06 - acc: 1.000 - ETA: 5s - loss: 1.9454e-06 - acc: 1.000 - ETA: 5s - loss: 1.9067e-06 - acc: 1.000 - ETA: 5s - loss: 1.9159e-06 - acc: 1.000 - ETA: 5s - loss: 1.8994e-06 - acc: 1.000 - ETA: 5s - loss: 1.8875e-06 - acc: 1.000 - ETA: 5s - loss: 1.8433e-06 - acc: 1.000 - ETA: 5s - loss: 1.8374e-06 - acc: 1.000 - ETA: 4s - loss: 1.8095e-06 - acc: 1.000 - ETA: 4s - loss: 1.7882e-06 - acc: 1.000 - ETA: 4s - loss: 1.7619e-06 - acc: 1.000 - ETA: 4

25000/25000 [==============================] - ETA: 6s - loss: 2.9087e-05 - acc: 1.000 - ETA: 3s - loss: 3.9564e-05 - acc: 1.000 - ETA: 3s - loss: 5.1998e-05 - acc: 1.000 - ETA: 3s - loss: 9.1947e-05 - acc: 1.000 - ETA: 3s - loss: 1.8278e-04 - acc: 1.000 - ETA: 3s - loss: 1.5744e-04 - acc: 1.000 - ETA: 3s - loss: 2.0180e-04 - acc: 1.000 - ETA: 3s - loss: 1.8498e-04 - acc: 1.000 - ETA: 3s - loss: 1.8378e-04 - acc: 1.000 - ETA: 3s - loss: 1.7846e-04 - acc: 1.000 - ETA: 3s - loss: 1.6667e-04 - acc: 1.000 - ETA: 3s - loss: 1.9059e-04 - acc: 1.000 - ETA: 3s - loss: 1.8259e-04 - acc: 1.000 - ETA: 3s - loss: 1.6853e-04 - acc: 1.000 - ETA: 3s - loss: 1.5733e-04 - acc: 1.000 - ETA: 3s - loss: 1.7003e-04 - acc: 1.000 - ETA: 2s - loss: 1.8679e-04 - acc: 1.000 - ETA: 2s - loss: 1.7441e-04 - acc: 1.000 - ETA: 2s - loss: 1.8076e-04 - acc: 1.000 - ETA: 2s - loss: 1.6999e-04 - acc: 1.000 - ETA: 2s - loss: 1.8666e-04 - acc: 1.000 - ETA: 2s - loss: 1.8006e-04 - acc: 1.000 - ETA: 2s - loss: 1.7228e-04 - 

25000/25000 [==============================] - ETA: 5s - loss: 1.9819e-06 - acc: 1.000 - ETA: 3s - loss: 1.0981e-06 - acc: 1.000 - ETA: 3s - loss: 1.1838e-06 - acc: 1.000 - ETA: 3s - loss: 1.3750e-06 - acc: 1.000 - ETA: 3s - loss: 1.3030e-06 - acc: 1.000 - ETA: 3s - loss: 1.2671e-06 - acc: 1.000 - ETA: 3s - loss: 1.2894e-06 - acc: 1.000 - ETA: 3s - loss: 1.2488e-06 - acc: 1.000 - ETA: 3s - loss: 1.2338e-06 - acc: 1.000 - ETA: 3s - loss: 1.2053e-06 - acc: 1.000 - ETA: 3s - loss: 1.1764e-06 - acc: 1.000 - ETA: 2s - loss: 1.1315e-06 - acc: 1.000 - ETA: 2s - loss: 1.0880e-06 - acc: 1.000 - ETA: 2s - loss: 1.1272e-06 - acc: 1.000 - ETA: 2s - loss: 1.1247e-06 - acc: 1.000 - ETA: 2s - loss: 1.1406e-06 - acc: 1.000 - ETA: 2s - loss: 1.1147e-06 - acc: 1.000 - ETA: 2s - loss: 1.1339e-06 - acc: 1.000 - ETA: 2s - loss: 1.1159e-06 - acc: 1.000 - ETA: 2s - loss: 1.1038e-06 - acc: 1.000 - ETA: 2s - loss: 1.1127e-06 - acc: 1.000 - ETA: 2s - loss: 1.1204e-06 - acc: 1.000 - ETA: 2s - loss: 1.1348e-06 - 

25000/25000 [==============================] - ETA: 5s - loss: 7.1712e-07 - acc: 1.000 - ETA: 3s - loss: 7.6856e-07 - acc: 1.000 - ETA: 3s - loss: 7.0364e-07 - acc: 1.000 - ETA: 3s - loss: 6.4544e-07 - acc: 1.000 - ETA: 3s - loss: 6.2242e-07 - acc: 1.000 - ETA: 3s - loss: 7.2501e-07 - acc: 1.000 - ETA: 3s - loss: 6.7390e-07 - acc: 1.000 - ETA: 3s - loss: 7.1479e-07 - acc: 1.000 - ETA: 3s - loss: 6.8456e-07 - acc: 1.000 - ETA: 3s - loss: 6.7374e-07 - acc: 1.000 - ETA: 3s - loss: 6.9070e-07 - acc: 1.000 - ETA: 3s - loss: 6.8384e-07 - acc: 1.000 - ETA: 3s - loss: 7.0262e-07 - acc: 1.000 - ETA: 3s - loss: 6.8885e-07 - acc: 1.000 - ETA: 3s - loss: 6.7743e-07 - acc: 1.000 - ETA: 3s - loss: 6.6900e-07 - acc: 1.000 - ETA: 2s - loss: 6.5200e-07 - acc: 1.000 - ETA: 2s - loss: 6.5262e-07 - acc: 1.000 - ETA: 2s - loss: 6.9280e-07 - acc: 1.000 - ETA: 2s - loss: 6.8841e-07 - acc: 1.000 - ETA: 2s - loss: 6.7974e-07 - acc: 1.000 - ETA: 2s - loss: 6.6920e-07 - acc: 1.000 - ETA: 2s - loss: 6.5701e-07 - 

25000/25000 [==============================] - ETA: 3s - loss: 6.0722e-07 - acc: 1.000 - ETA: 3s - loss: 4.0634e-07 - acc: 1.000 - ETA: 3s - loss: 3.9131e-07 - acc: 1.000 - ETA: 3s - loss: 3.8603e-07 - acc: 1.000 - ETA: 3s - loss: 3.7253e-07 - acc: 1.000 - ETA: 3s - loss: 3.7924e-07 - acc: 1.000 - ETA: 3s - loss: 3.7765e-07 - acc: 1.000 - ETA: 3s - loss: 3.8307e-07 - acc: 1.000 - ETA: 3s - loss: 3.8430e-07 - acc: 1.000 - ETA: 3s - loss: 3.8188e-07 - acc: 1.000 - ETA: 3s - loss: 3.8861e-07 - acc: 1.000 - ETA: 2s - loss: 4.0054e-07 - acc: 1.000 - ETA: 2s - loss: 3.9419e-07 - acc: 1.000 - ETA: 2s - loss: 4.0462e-07 - acc: 1.000 - ETA: 2s - loss: 4.3906e-07 - acc: 1.000 - ETA: 2s - loss: 4.3875e-07 - acc: 1.000 - ETA: 2s - loss: 4.3547e-07 - acc: 1.000 - ETA: 2s - loss: 4.3546e-07 - acc: 1.000 - ETA: 2s - loss: 4.4521e-07 - acc: 1.000 - ETA: 2s - loss: 4.3947e-07 - acc: 1.000 - ETA: 2s - loss: 4.3112e-07 - acc: 1.000 - ETA: 2s - loss: 4.3312e-07 - acc: 1.000 - ETA: 2s - loss: 4.3868e-07 - 

In [15]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)
